<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [2]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [3]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [4]:
import spacy
from spacy.tokenizer import Tokenizer
import re

In [5]:
nlp = spacy.load('en_core_web_lg')
tokenizer = Tokenizer(nlp.vocab)

In [6]:
tokenizer.pipe(yelp['text'])

In [7]:
def tokenize(doc):
    
    doc_tokens = []
    
    for item in tokenizer(doc):
    
#         for token in item:
            if (item.is_stop == False) & (item.is_punct == False):
                lowers = re.sub('[^a-zA-z 0-9]', '', item.lemma_).lower()
                if lowers not in nlp.Defaults.stop_words:
                    doc_tokens.append(lowers)

    return doc_tokens

In [ ]:
tokenize(yelp['text'][0])

In [8]:
yelp['tokens'] = yelp['text'].apply(tokenize)
yelp[['tokens']].head()

,tokens
0,"[beware, fake, fake, fakewe, small, business, ..."
1,"[came, lunch, togo, service, quick, staff, fri..."
2,"[ive, vegas, dozen, time, step, foot, circus, ..."
3,"[night, close, street, party, actually, group,..."
4,"[35, 4, star, , bad, price, 1299, lunch, senio..."


## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [10]:
def token_string(token_list):
    string0 = ''
    for token in token_list:
        string0 = string0 + ' ' + token
    return string0

# Photo credit: Will Stauffer-Norris

In [38]:
token_string(yelp['text'][0])

'BEWARE!!! FAKE, FAKE, FAKE....We also own a small business in Los Alamitos, CA and received what looked like a legitimate bill for $70 with an account number and all.  I called the phone number listed (866) 273-7934.  The wait time on hold said 20 minutes and to leave a message.  I could not get a live person on the phone no matter what number I selected.  I left a very FIRM message that I would be contacting the BBB and my attorney regarding their company trying to scam businesses. This has to be illegal!!!!!'

In [11]:
yelp['token_string'] = yelp['tokens'].apply(token_string)
yelp[['token_string']].head()

,token_string
0,beware fake fake fakewe small business los al...
1,came lunch togo service quick staff friendly ...
2,ive vegas dozen time step foot circus circus ...
3,night close street party actually group guy p...
4,35 4 star bad price 1299 lunch senior 15 p...


In [12]:
tfidf = TfidfVectorizer(
    stop_words='english',
    ngram_range=(1,2),
    max_df=.99,
    min_df=.001
)

dtm = tfidf.fit_transform(yelp['token_string'])

dtm_df = pd.DataFrame(
    dtm.todense(), columns=tfidf.get_feature_names()
)

dtm_df.head()

,10,10 min,10 minute,10 minutes,10 people,10 year,10 years,100,1000,101,...,yuck,yum,yummy,yup,zero,zero star,zero stars,zone,zoo,zucchini
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
nn.fit(dtm_df)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [14]:
nn.kneighbors([dtm_df.iloc[8]])

(array([[0.        , 0.94950629, 0.99157227, 1.        , 1.        ]]),
 array([[   8, 9196,  658, 6311, 6204]], dtype=int64))

In [15]:
yelp['text'][8][:400]

"Absolutely the most Unique experience in a nail shop far as Charlotte,Nc is concerned! First i must start with i felt like a queen while getting my pedicure they have these gorgeous and glamourous high end chairs they seat you in for your pedicure.May i add i haven't had a good foot massage in a while at any nail shop and they nailed it i darn near fell asleep. If you ever want to splurge while yo"

In [16]:
yelp['text'][9196][:400]

"I went to a nail salon and they really messed up my nails uneven and we're ugly. Went to elegant nails and he fixed my nails and they are beautiful now and my pedicure was amazing .Tran is the best... best pedicure I've ever  had ...Thank You"

In [17]:
hypothetical_review = ["""
    Most of the time when I go through the drive through,
    they tell me the ice cream machine is broken. But
    when I order inside, I've always gotten ice cream.
    I'm starting to think that this is a conspiricy.
"""]

In [18]:
rev_mtrx = tfidf.transform(hypothetical_review)

In [19]:
rev_vec = rev_mtrx.todense()

In [20]:
nn.kneighbors(rev_vec)

(array([[0.79874436, 0.91448563, 0.9306582 , 0.95585238, 0.96313871]]),
 array([[6700, 9632,   41, 4075, 9975]], dtype=int64))

In [21]:
yelp['text'][6700][:400]

'Fun experience! \n\nThey have various ice cream creations that they will mash and mix up for you so you can watch. Its cool how the ice cream hardens so fast and they are able to create rolled up ice cream. \n\nIce cream tastes just like norm ice cream but the idea and how they make it different is really neat!'

In [22]:
yelp['text'][9632][:400]

'I am no fan sweets so it takes quite a bit to impress my palate. Although, I do have a soft spot for the ice cream sandwiches at Bakerbots/ Bang Bang in the west side of town. So the stage is set for a ice cream sandwich throwdown!\n\nI had their salted caramel cookie with vanilla ice cream and it was heavenly for the most part. I love the cookie; perfectly sweet, perfectly salty with a pinch of cha'

In [61]:
# The pattern is the ice cream shop. Same with the reviews
# of the nail salons. It is consistent in the type of
# establishment that it notices a pattern from. However, the
# algorithm did not pick up on my sentiment of suspicion...

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [23]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [24]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier()

In [25]:
svd = TruncatedSVD(
    n_components=100,
    algorithm='randomized',
    n_iter=10
)

lsi = Pipeline([('vect',vect),('svd',svd)])
vect = TfidfVectorizer(stop_words='english')

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

In [28]:
parameters = {
    'lsi__svd__n_components': [100], #[10,100,250],
    'lsi__vect__max_df': (0.75, 1.0),
    'clf__max_depth': [10] #(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-2, verbose=1)
grid_search.fit(yelp['token_string'], yelp['stars'])

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  10 out of  10 | elapsed:  6.6min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [29]:
grid_search.predict(hypothetical_review)

array([5], dtype=int64)

In [30]:
# Eww. Sorely inaccurate. I probably wouldn't give such a suspicious
# Ice cream shop any rating above a 3.

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [31]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

Learn the vocubalary of the yelp data:

In [34]:
id2word = Dictionary(yelp['tokens'])
print(len(id2word))

29734


In [35]:
id2word.filter_extremes(no_below=5, no_above=.98)
print(len(id2word))

6715


Create a bag of words representation of the entire corpus

In [36]:
corpus = [id2word.doc2bow(d) for d in yelp['tokens']]

Your LDA model should be ready for estimation: 

In [38]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=3,
                   num_topics = 10 # You can change this parameter
                  )

In [39]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [42]:
topics = [' '.join(t[0:10]) for t in words]

In [43]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
   good great place service like food time come

------ Topic 1 ------
   place great food time come like service good

------ Topic 2 ------
   place food great good come like time order

------ Topic 3 ------
   service place food like time great try good

------ Topic 4 ------
   good place great food time like order service

------ Topic 5 ------
   time place food great service good order come

------ Topic 6 ------
   good food time come like service order place

------ Topic 7 ------
   food come place good order like service great

------ Topic 8 ------
   place good food like time service love great

------ Topic 9 ------
   food place like great time come service order



In [44]:
distro = [lda[d] for d in corpus]
num_topics = 20

def update(doc):
        d_dist = {k:0 for k in range(0,num_topics)}
        for topic in doc:
            d_dist[topic[0]] = topic[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

Create 1-2 visualizations of the results

In [45]:
import pyLDAvis.gensim

In [46]:
pyLDAvis.enable_notebook()

In [48]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.003899 -0.004103       1        1  10.269718
8      0.003409 -0.001081       2        1  10.215357
4      0.001676  0.003166       3        1  10.194074
2     -0.003531 -0.001286       4        1  10.143779
6      0.000896 -0.003708       5        1  10.125352
9     -0.004487  0.000872       6        1  10.089141
0      0.003143  0.004898       7        1   9.940676
5      0.000497 -0.000035       8        1   9.906551
3      0.000529  0.001431       9        1   9.615592
1     -0.006031 -0.000155      10        1   9.499761, topic_info=          Term          Freq         Total Category  logprob  loglift
0               15174.000000  15174.000000  Default  30.0000  30.0000
49              12086.000000  12086.000000  Default  29.0000  29.0000
39        good   4711.000000   4711.000000  Default  28.0000  28.0000
195      great   4262.000000   4262.000000  Default  27.0000  27.0000
122      place   5212.000000   5212.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
173  recommend    126.312668   1314.304077  Topic10  -5.8800   0.0116
388        ask    129.568848   1480.725586  Topic10  -5.8545  -0.0822
34         try    150.406494   2461.367920  Topic10  -5.7054  -0.4412
88        nice    133.955536   1745.505249  Topic10  -5.8212  -0.2134
426       know    130.258392   1615.326416  Topic10  -5.8492  -0.1639

[849 rows x 6 columns], token_table=      Topic      Freq Term
term                      
49        1  0.107480     
49        2  0.099702     
49        3  0.083319     
49        4  0.135860     
49        5  0.103094     
...     ...       ...  ...
5933      6  0.079350    z
5933      7  0.158700    z
5933      8  0.079350    z
5933      9  0.079350    z
5933     10  0.079350    z

[4887 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 9, 5, 3, 7, 10, 1, 6, 4, 2])

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)